In [56]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
import pandas as pd
import numpy as np
np.random.seed(132)
from functools import lru_cache

import sys

CODE_PATH = '../code'

sys.path.append(CODE_PATH)
import functions

from sklearn.linear_model import LogisticRegression

from scipy.optimize import minimize
from tqdm import tqdm
%pylab inline

def sq(a):
    return np.dot(a, a)

Populating the interactive namespace from numpy and matplotlib


# Load Data

In [68]:
from sklearn.datasets import load_linnerud, load_iris, load_boston, load_breast_cancer, load_wine, load_digits

dataset = load_wine()
# dataset = load_boston()
# dataset = load_breast_cancer()
# dataset = load_digits()

df = pd.DataFrame(dataset['data'])
target = dataset['target']
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


In [69]:
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

In [ ]:
df = pd.read_csv('../data/abalone.csv', header=None)
df = df.drop(0, 1)

In [70]:
# НОРМИРОВКА НА СРЕДНЕЕ, МОЖНО ОПУСТИТЬ ИНОГДА
df = (df - df.mean())/(df.max() - df.min())

In [71]:
df0 = df.copy()
print(df.shape)
df.head()

(178, 13)


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.323522,-0.123784,0.033948,-0.200770,0.296287,0.174099,0.217454,-0.154441,0.220537,0.049651,0.067114,0.479236,0.226895
1,0.052469,-0.109950,-0.121132,-0.427574,0.002809,0.122375,0.154163,-0.192177,-0.098075,-0.057857,0.075244,0.288760,0.216196
2,0.041943,0.004674,0.162290,-0.046131,0.013679,0.174099,0.255428,-0.116706,0.384574,0.053064,0.058984,0.204511,0.312487
3,0.360364,-0.076353,0.071381,-0.138915,0.144113,0.536168,0.308171,-0.229913,0.185836,0.233951,-0.079227,0.307075,0.522901
4,0.062995,0.050129,0.269242,0.077580,0.198461,0.174099,0.139395,0.053106,0.072272,-0.062977,0.067114,0.116599,-0.008483


## Insert NaNs

In [72]:
df, df1, df2, arr_nan, arr_raw, arr_known, arr_pred, nan_coords = functions.set_nans(df0, 180, 5, 0.2)

Num nan places: 175
(178, 8) (178, 13)


In [73]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score


def run_cv(X, y, clf, num_folds):
    skf = StratifiedKFold(n_splits=num_folds)
    scores = []
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf.fit(X_train, y_train)
        acc = clf.score(X_test, y_test) # accuracy
        scores.append(acc)
    scores = np.array(scores)
    return scores.mean(), scores.std()

In [74]:
clf = LogisticRegression()
def iterative_process(update_function, max_iter=20, verbose=True, percentage=False):
    df_result = pd.DataFrame(np.zeros((max_iter, 4)), columns=['mae', 'rmse', 'acc_mean', 'acc_std'])
    
    for i in range(max_iter):
        update_function()
        if percentage:
            mae = functions.get_mape(arr_raw, arr_pred, nan_coords)
            rmse = functions.get_rmspe(arr_raw, arr_pred, nan_coords)
        else:
            mae = functions.get_mae(arr_raw, arr_pred, nan_coords)
            rmse = functions.get_rmse(arr_raw, arr_pred, nan_coords)
        df_result.loc[i, 'mae'] = mae
        df_result.loc[i, 'rmse'] = rmse
        acc_mean, acc_std = run_cv(arr_pred, target, clf, 5)
        if verbose:
            print('\tIteration {}  mae {:.6} // rmse {:.6}\
                // mean {:.6} // std {:.6}'.format(i+1, mae, rmse, acc_mean, acc_std))
    return df_result

In [75]:
mae = functions.get_mae(arr_raw, arr_pred, nan_coords)
rmse = functions.get_rmse(arr_raw, arr_pred, nan_coords)
print(mae, rmse)
acc_mean, acc_std = run_cv(arr_pred, target, clf, 5)
print(acc_mean, acc_std)

0.18337418111715287 0.22093424587751495
0.9615529815529815 0.03648570489532294


# Local Approach

In [76]:
from sklearn.neighbors import NearestNeighbors
k_local = 15
teta = 1
nbrs = NearestNeighbors(n_neighbors=k_local, algorithm='ball_tree').fit(arr_pred)

def get_delta_vec(teta):
    deltas = []
    _, heighbours = nbrs.kneighbors(arr_pred)
    for i, j in nan_coords:
        mean_neigh = arr_pred[heighbours[i]].mean(axis=0)[j]
        deltas.append(teta*(mean_neigh - arr_pred[i][j]))
    return np.array(deltas)

def update_local():
    deltas = get_delta_vec(teta)
    nbrs = NearestNeighbors(n_neighbors=k_local, algorithm='ball_tree').fit(arr_pred) # пересчитываем
    for j, (x,y) in enumerate(nan_coords):
        arr_pred[x, y] += deltas[j]

In [77]:
dfr = iterative_process(update_local, max_iter=30, verbose=True)

	Iteration 1  mae 0.125412 // rmse 0.168432                // mean 0.956148 // std 0.0469167
	Iteration 2  mae 0.105615 // rmse 0.151534                // mean 0.961862 // std 0.0500642
	Iteration 3  mae 0.0988251 // rmse 0.146109                // mean 0.967267 // std 0.0396823
	Iteration 4  mae 0.0958399 // rmse 0.142911                // mean 0.967267 // std 0.0396823
	Iteration 5  mae 0.0946123 // rmse 0.141827                // mean 0.967267 // std 0.0396823
	Iteration 6  mae 0.0944987 // rmse 0.1417                // mean 0.967267 // std 0.0396823
	Iteration 7  mae 0.0949091 // rmse 0.142328                // mean 0.967267 // std 0.0396823
	Iteration 8  mae 0.0947061 // rmse 0.142232                // mean 0.967267 // std 0.0396823
	Iteration 9  mae 0.0946778 // rmse 0.142224                // mean 0.967267 // std 0.0396823
	Iteration 10  mae 0.0947049 // rmse 0.142242                // mean 0.967267 // std 0.0396823
	Iteration 11  mae 0.0946756 // rmse 0.142226                //

In [ ]:
k_local/len(df)

# Optimization Approach

In [78]:
n = len(df0)

def get_known_table():
    global arr_nan
    ro_known_table = np.zeros((n, n))
    for i in tqdm(range(n)):
        for j in range(i, n):
            idx_nan_i = np.isnan(arr_nan[i])
            idx_nan_j = np.isnan(arr_nan[j])
            idx = ~idx_nan_i & ~idx_nan_j
            x_i = arr_nan[i, idx]
            x_j = arr_nan[j, idx]
            ro_known_table[i, j] = ro_known_table[j, i] = sq(x_i - x_j)
    return ro_known_table

def get_ro_1_unknown_table():
    global arr_pred
    ro_1_unknown_table = np.zeros((n, n))
    #это ro прибавить там где 1 компонента
    for i in tqdm(range(n)):
        for j in range(i, n):
            idx_nan_i = np.isnan(arr_nan[i])
            idx_nan_j = np.isnan(arr_nan[j])
            idx = (idx_nan_i & ~idx_nan_j) | (~idx_nan_i & idx_nan_j)
            x_i = arr_pred[i, idx]
            x_j = arr_pred[j, idx]
            ro_1_unknown_table[i, j] = ro_1_unknown_table[j, i] = sq(x_i - x_j)
    return ro_1_unknown_table

# по всем признакам
def get_ro_2_unknown_table():
    global arr_pred
    '''
    при первом запуске обычно 0, так как заполняю пропуски ОДНИМ И ТЕМ ЖЕ СРЕДНИМ ЗНАЧЕНИЕМ
    '''
    ro_2_unknown_table = np.zeros((n, n))
    for i in tqdm(range(n)):
        for j in range(i+1, n):
            idx_nan_i = np.isnan(arr_nan[i])
            idx_nan_j = np.isnan(arr_nan[j])
            idx = (idx_nan_i & idx_nan_j)
            x_i = arr_pred[i, idx]
            x_j = arr_pred[j, idx]
            ro_2_unknown_table[i, j] = ro_2_unknown_table[j, i] = sq(x_i - x_j)
    return ro_2_unknown_table

In [79]:
def F_functional(k_arr, ftype='f1'):
    global ro_known_table, ro_1_unknown_table, ro_2_unknown_table
    f = 0
    for i in tqdm(range(n)):
        for j in range(i+1, n):
            ro1 = ro_known_table[i, j] # известно оба 
            ro2 = ro_1_unknown_table[i, j] # известен 1 
            ro3 = ro_2_unknown_table[i, j] # не известно ниче
            
            r1 = np.sqrt(ro1 + ro2 + ro3) #по всем признакам(р)
            if ftype == 'f1':
                r2 = np.sqrt(ro1) # по известным признакам(р+)
            else:
                r2 = np.sqrt(ro1 + ro2)
            f += (r1 - k_arr[i, j]*r2)**2
    return f

In [80]:
### ПРОВЕРИТЬ ГРАДИЕНТ НА 0 ДЛЯ Ф2

def grad_component(i, j, k_arr, ftype='f1'):
    global ro_known_table, ro_1_unknown_table, ro_2_unknown_table
    s = 0
    for i1 in range(n):
        if i1 == i:
            continue
        ro1 = ro_known_table[i, i1]
        ro2 = ro_1_unknown_table[i, i1]
        ro3 = ro_2_unknown_table[i, i1]
        
        r1 = np.sqrt(ro1 + ro2 + ro3) #по всем признакам(р)
        if r1 <= 0:
            continue
        if ftype == 'f2':
            r2 = np.sqrt(ro1 + ro2) # по известным признакам(р+)
            delta_x1 = (arr_pred[i, j] - arr_pred[i1, j])
            if not np.isnan(arr_nan[i1, j]):
                delta_x2 = delta_x1
            else:
                delta_x2 = 0
            if r2 == 0:
                delta_x2 = k_arr[i1, i]
            else:
                delta_x2 = k_arr[i1, i]*delta_x2/r2
            delta_s = 2*(r1 - k_arr[i1, i]*r2)*(delta_x1/r1 - delta_x2)
            s += delta_s
        else:
            r2 = np.sqrt(ro1) # по известным признакам(р+)
            s += 2 * (1 - k_arr[i1, i]*r2/r1) * (arr_pred[i, j] - arr_pred[i1, j])
    return s

def get_full_grad(nan_coords, k_arr, ftype='f1'):
    grad = []
    for i, j in nan_coords:
        grad.append(grad_component(i, j, k_arr, ftype))
    return np.array(grad)

In [81]:
df, df1, df2, arr_nan, arr_raw, arr_known, arr_pred, nan_coords = functions.set_nans(df0, 180, -1, 0.2)

Num nan places: 462
(178, 0) (178, 13)


In [82]:
ro_known_table = get_known_table()
ro_1_unknown_table = get_ro_1_unknown_table()
ro_2_unknown_table = get_ro_2_unknown_table() # здесь какого-то хера одни нули!!!

100%|██████████| 178/178 [00:00<00:00, 1355.84it/s]


In [83]:
k_arr = np.ones((n, n))
ftype = 'f1'
alpha = 0.005
alpha = 0.001
c = F_functional(k_arr, ftype)

100%|██████████| 178/178 [00:00<00:00, 2471.98it/s]


In [ ]:
c = F_functional(k_arr, ftype)

In [ ]:
# Для F2 нужно делать не единички, а нормировать
# k_arr[i, j] = n - n2 / n (то есть все признаки без двух пропусков/на все признаки)
k_arr = np.ones((n, n))
alpha = 1.1
for i in tqdm(range(n)):
    for j in range(i+1, n):
        idx_nan_i = np.isnan(arr_nan[i])
        idx_nan_j = np.isnan(arr_nan[j])
        idx = (idx_nan_i & idx_nan_j)
        k_arr[i, j] = k_arr[j, i] = (n - idx.sum()) / n
ftype = 'f2'
c = F_functional(k_arr, ftype)

In [84]:
# проверка изменений в зависимости от альфы
arr_pred_raw = arr_pred.copy()
grad = get_full_grad(nan_coords, k_arr, ftype)

def get_f_grad(arr_pred_raw, alpha, grad):
    global ro_1_unknown_table, ro_2_unknown_table
    for j, (x,y) in enumerate(nan_coords):
        arr_pred[x, y] = arr_pred_raw[x, y] - alpha*grad[j]
    ro_1_unknown_table = get_ro_1_unknown_table()
    ro_2_unknown_table = get_ro_2_unknown_table()
    c = F_functional(k_arr, ftype)
    return c

In [85]:
c_array = []

ro1t = []
ro2t = []
def update_opt():
    global ro_1_unknown_table, ro_2_unknown_table
    grad = get_full_grad(nan_coords, k_arr, ftype)
    for j, (x,y) in enumerate(nan_coords):
        arr_pred[x, y] = arr_pred[x, y] - alpha*grad[j]
        
    ro_1_unknown_table = get_ro_1_unknown_table()
    ro_2_unknown_table = get_ro_2_unknown_table()
    c = F_functional(k_arr, ftype)
    print('C = {}'.format(c))
    c_array.append(c)
    ro1t.append(ro_1_unknown_table)
    ro2t.append(ro_2_unknown_table)

In [86]:
max_iter = 30

dfr = iterative_process(update_opt, max_iter=max_iter, verbose=True)

100%|██████████| 178/178 [00:00<00:00, 2407.69it/s]


C = 325.2134067451456
	Iteration 1  mae 0.159912 // rmse 0.198382                // mean 0.95537 // std 0.012783


100%|██████████| 178/178 [00:00<00:00, 2383.11it/s]


C = 323.1313871153268
	Iteration 2  mae 0.15871 // rmse 0.197089                // mean 0.95537 // std 0.012783


100%|██████████| 178/178 [00:00<00:00, 3438.38it/s]


...

C = 312.05371315329796
	Iteration 28  mae 0.147862 // rmse 0.185294                // mean 0.95537 // std 0.012783


100%|██████████| 178/178 [00:00<00:00, 3648.27it/s]


C = 312.0250896260559
	Iteration 29  mae 0.147762 // rmse 0.185186                // mean 0.95537 // std 0.012783


100%|██████████| 178/178 [00:00<00:00, 4033.81it/s]


C = 312.0002140370237
	Iteration 30  mae 0.14767 // rmse 0.185086                // mean 0.95537 // std 0.012783


In [ ]:
print(c_array, ftype)

for i in range(7):
    print((ro2t[i] - ro2t[i+1]).max())

# SNE Approach

In [87]:
df, df1, df2, arr_nan, arr_raw, arr_known, arr_pred, nan_coords = functions.set_nans(df0, 180, -1, 0.2)

Num nan places: 462
(178, 0) (178, 13)


In [88]:
n = len(df0)
# таблица старых значений
Exp1_table = np.zeros((n, n))
for i in tqdm(range(n)):
    for j in range(n):
        Exp1_table[i][j] = np.exp(-sq(arr_known[i] - arr_known[j]))
#         Exp1_table[i][j] = np.exp(-norm(arr_known[i] - arr_known[j])**2)
        
P1_table = np.zeros((n, n))
for i in tqdm(range(n)):
    for j in range(n):
        a = Exp1_table[j][i]
        b = Exp1_table[i].sum() - 1 # 1 = Exp1_table[i][i]
        P1_table[j][i] = a / b

100%|██████████| 178/178 [00:00<00:00, 2006.40it/s]


In [89]:
def get_exp2_table():
    Exp2_table = np.zeros((n, n))
    for i in tqdm(range(n)):
        for j in range(n):
            Exp2_table[i][j] = np.exp(-sq(arr_pred[i] - arr_pred[j]))
#             Exp2_table[i][j] = np.exp(-norm(arr_pred[i] - arr_pred[j])**2)
    return Exp2_table

def get_p2_table():
    Exp2_table = get_exp2_table()
    P2_table = np.zeros((n, n))
    for i in tqdm(range(n)):
        for j in range(n):
            a = Exp2_table[j][i]
            b = Exp2_table[i].sum() - 1 # 1 = Exp1_table[i][i]
            P2_table[j][i] = a / b
    return P2_table
P2_table = get_p2_table()

100%|██████████| 178/178 [00:00<00:00, 2183.68it/s]


In [90]:
def KL(P2_table):
    global P1_table
#     global P1_table, P2_table
    s = 0
    for i in range(n):
        for j in range(n):
#             s += P1_table[j][i] * np.log(P1_table[j][i] / P2_table[j][i])
            s += P1_table[j][i] * np.log(P1_table[j][i]) - P1_table[j][i] * np.log(P2_table[j][i])
    return s

def get_grad_sne(i1, i2):
    global P1_table, P2_table
    def get_i_part(i):
        d = 2*(arr_pred[i1][i2] - arr_pred[i][i2])
        s = (P1_table[i1][i]+P1_table[i][i1]) - \
            P2_table[i1][i]*(1+P1_table[i][i]) - \
            P2_table[i][i1]*(1+P1_table[i1][i1])
        return s*d
    return sum(get_i_part(i) for i in range(n) if i!=i1)

def get_full_grad_sne(nan_coords):
    n_gaps = len(nan_coords)
    grad = np.zeros(n_gaps)
    for i in range(n_gaps):
        i1, i2 = nan_coords[i]
        grad[i] = get_grad_sne(i1, i2)
    return grad

In [91]:
momentum = True
alpha_momentum = 0.5
c_array = []
p_array = []
g_array = []

y_momentum = np.zeros(len(nan_coords))
for j, (x,y) in enumerate(nan_coords):
    y_momentum[j] = arr_pred[x, y]
y_array = [y_momentum]

def update_sne():
    grad = get_full_grad_sne(nan_coords)

    for j, (x,y) in enumerate(nan_coords):
        arr_pred[x, y] = arr_pred[x, y] - alpha*grad[j]
    # если есть еще предыдущие, то делаем шаг моментум
    if momentum and len(y_array) >= 2:
        if len(y_array) >= 2:
            y_diff = y_array[-1] - y_array[-2]
            for j, (x,y) in enumerate(nan_coords):
                arr_pred[x, y] += alpha_momentum*y_diff[j]
        y_momentum = np.zeros(len(nan_coords))
        for j, (x,y) in enumerate(nan_coords):
            y_momentum[j] = arr_pred[x, y]
        y_array.append(y_momentum)
    
    P2_table = get_p2_table()
    c = KL(P2_table)
    print('C = {}'.format(c))
    c_array.append(c)
    p_array.append(P2_table)
    g_array.append(grad)

In [93]:
alpha = 0.05
dfr = iterative_process(update_sne, max_iter=150, verbose=True)

100%|██████████| 178/178 [00:00<00:00, 2117.25it/s]


C = 13.217365555950137
	Iteration 1  mae 0.164083 // rmse 0.202859                // mean 0.955212 // std 0.0132681


100%|██████████| 178/178 [00:00<00:00, 2851.73it/s]


C = 13.038964600590857
	Iteration 2  mae 0.166942 // rmse 0.205935                // mean 0.955212 // std 0.0132681


100%|██████████| 178/178 [00:00<00:00, 2050.98it/s]


...

C = 10.777957800806515
	Iteration 25  mae 0.235596 // rmse 0.280572                // mean 0.883104 // std 0.0383695


100%|██████████| 178/178 [00:00<00:00, 2269.07it/s]


C = 10.739162410355572
	Iteration 26  mae 0.238627 // rmse 0.283925                // mean 0.883104 // std 0.0383695


100%|██████████| 178/178 [00:00<00:00, 2025.12it/s]


KeyboardInterrupt: 

In [ ]:
s = 0
for i in range(n):
    for j in range(n):
        s += P1_table[j][i] * np.log(P1_table[j][i] / P2_table[j][i])
    print(s)
return s

# AVO APPROACH
Единственный подход, который не содержит оптимизайионный подход

In [48]:
df, df1, df2, arr_nan, arr_raw, arr_known, arr_pred, nan_coords = functions.set_nans(df0, 180, -1, 0.2)

Num nan places: 23001
(1797, 0) (1797, 64)


In [49]:
def get_epsilons(data):
    def get_e(a):
        return np.abs(a - a[:, None]).mean()
    return np.array([get_e(feat) for feat in data.T])

In [50]:
def get_gamma(data, x, k, epsilons=None):
    if epsilons is not None:
        Gamma = 0
        for x_i in data:
            d = (np.abs(x_i - x) < epsilons).sum()
            Gamma += functions.Cnk(d, k)
        return Gamma
    # расстояния между парами объектов   
    n = data.shape[0]
    n_feat = data.shape[1]
    maps = np.zeros(data.T.shape)
    for i in range(n_feat):
        maps[i] = data.T[i] <= x[i] 
    maps = maps.T
    Gamma = 0
    for i in tqdm(range(n)):
        for j in range(i, n):
            d = (maps[i] + maps[j] == 1).sum()
            Gamma += functions.Cnk(d, k)
    return Gamma*2/(n*(n-1))

In [51]:
def predict(data, y, x, k, epsilons=None, scores=False):
    # WTF ZERO
    n_class = max(y) + 1
    g_classes = np.zeros(n_class)
    y = y.values 
    for i in range(n_class):
        c_data = data[y == i]
        g_classes[i] = get_gamma(c_data, x, k, epsilons)
    if scores:
        return g_classes
    return np.argmax(g_classes)

def predict_vect(X_train, y_train, X_test, k, epsilons=None, scores=False):
    return np.array([predict(X_train, y_train, x, k, epsilons, scores) for x in X_test])

In [52]:
# итеративный процесс
def solve_avo(df, x, y):
    x_mask = df.iloc[:, y].isnull() 
    x_mask = x_mask[~x_mask].index # маска по объектам для трейна
    y_mask = df.isnull().sum() > 0 
    y_mask = y_mask[~y_mask].index # маска по по признакам для трейна/теста
    
    X_train = df.iloc[x_mask, y_mask]
    X_test = df.iloc[x, y_mask].values.reshape(1, -1)

    y_train_raw = df.iloc[x_mask, y]
    y_train = y_train_raw.rank(method='dense').astype(int) - 1
    
    eps = get_epsilons(X_train.values)
    gammas_ki = predict(X_train.values, y_train, X_test, 3, eps, True)
    sizes_ki = y_train.value_counts().sort_index().values
    
    full_range = np.array(range(sizes_ki.size))
    sort_index = solve_avo_gap(gammas_ki, sizes_ki, full_range)
    return y_train_raw.sort_values().iloc[sort_index]
    
def solve_avo_gap(gammas_ki, sizes_ki, full_range):
#     print(full_range)
    if len(full_range) == 1:
        return full_range[0]
    left_range = full_range[:full_range.size//2]
    right_range = full_range[full_range.size//2:]
    left_score = get_gamma_range(gammas_ki, sizes_ki, left_range)
    right_score = get_gamma_range(gammas_ki, sizes_ki, right_range)
    if left_score < right_score:
        return solve_avo_gap(gammas_ki, sizes_ki, right_range)
    else:
        return solve_avo_gap(gammas_ki, sizes_ki, left_range)
        
def get_gamma_range(gammas_ki, sizes_ki, y_range):
    sum_gammas = gammas_ki[y_range].sum()
    sum_sizes = sizes_ki[y_range].sum()
    if sum_sizes == 0:
        return 0
    return sum_gammas/sum_sizes

In [54]:
# по всем пропускам из таблицы запускаем и проходим
for x, y in tqdm(nan_coords):
    arr_pred[x, y] = solve_avo(df, x, y)

100%|██████████| 23001/23001 [04:47<00:00, 80.09it/s]


In [ ]:
# for x, y in tqdm(nan_coords):
#     print(arr_pred[x, y])

In [55]:
mae = functions.get_mae(arr_raw, arr_pred, nan_coords)
rmse = functions.get_rmse(arr_raw, arr_pred, nan_coords)
print(mae, rmse)
acc_mean, acc_std = run_cv(arr_pred, target, clf, 5)
print(acc_mean, acc_std)

4.902482500760836 7.769471011370761
0.8041373328622606 0.034992694930339845


In [ ]:

7.62472820045 14.1056553537